In [29]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split

## Task_1. Попробуйте видоизменить параметры разобранной на уроке нейронной сети таким образом, чтобы улучшить её точность. Проведите анализ:
- Что приводит к ухудшению точности нейронной сети?
- Что приводит к увеличению её точности?

In [24]:
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид
def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [11]:
iris_data = pd.read_csv("./Iris.csv")
iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# формирование входных данных
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns=columns)
x = normalize(x.values)

# формирование выходных данных(результатов)
columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.values
y = y.flatten()
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [32]:
def neural_net(neuron_number=2,learning_rate=10**-3,num_of_epochs=5):    
    
    errors = []
    neuron_number = neuron_number
    n = learning_rate
    
    #инициализация весов
    w0 = 2*np.random.random((4, neuron_number)) - 1 # для входного слоя   - 4 входа, n выхода
    w1 = 2*np.random.random((neuron_number, 3)) - 1 # для внутреннего слоя - n входов, 3 выхода

    # процесс обучения
    for _ in range(num_of_epochs):
        for i in range(10000): 

            # прямое распространение(feed forward)
            layer0 = X_train
            layer1 = sigmoid(np.dot(layer0, w0))
            layer2 = sigmoid(np.dot(layer1, w1))

            # обратное распространение(back propagation) с использованием градиентного спуска
            layer2_error = y_train - layer2
            layer2_delta = layer2_error * sigmoid_deriv(layer2)

            layer1_error = layer2_delta.dot(w1.T)
            layer1_delta = layer1_error * sigmoid_deriv(layer1)

            w1 += layer1.T.dot(layer2_delta) * n
            w0 += layer0.T.dot(layer1_delta) * n

            error = np.mean(np.abs(layer2_error))
            errors.append(error)
    accuracy = (1 - error) * 100

    print(f'Model with {neuron_number} neurons, {n}-learning rate and {num_of_epochs} epochs has an accuracy of {round(accuracy,2)}.')

In [30]:
neuron_numbers = [1,2,3,4]
learning_rate = [10**-3,10**-2,10**-1,10**0]
num_of_epochs = [1,2,3,4]

In [31]:
print('*************\nChanging num. of neurons\n*************')
for i in range(4):
    neural_net(neuron_numbers[i])
    
print('*************\nChanging learning rate\n*************')
for i in range(4):
    neural_net(neuron_numbers[0],learning_rate[i])

print('*************\nChanging num. of epochs\n*************')
for i in range(4):
    neural_net(neuron_numbers[0],learning_rate[0],num_of_epochs[i])

*************
Changing num. of neurons
*************
Model with 1 neurons, 0.001-learning rate and 5 epochs has an accuracy of 57.28.
Model with 2 neurons, 0.001-learning rate and 5 epochs has an accuracy of 82.11.
Model with 3 neurons, 0.001-learning rate and 5 epochs has an accuracy of 86.91.
Model with 4 neurons, 0.001-learning rate and 5 epochs has an accuracy of 90.81.
*************
Changing learning rate
*************
Model with 1 neurons, 0.001-learning rate and 5 epochs has an accuracy of 57.34.


C:\Users\Ivano\AppData\Local\Temp\ipykernel_5344\3538275795.py:21: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Model with 1 neurons, 0.01-learning rate and 5 epochs has an accuracy of 58.11.
Model with 1 neurons, 0.1-learning rate and 5 epochs has an accuracy of 57.82.
Model with 1 neurons, 1-learning rate and 5 epochs has an accuracy of 50.0.
*************
Changing num. of epochs
*************
Model with 1 neurons, 0.001-learning rate and 1 epochs has an accuracy of 56.56.
Model with 1 neurons, 0.001-learning rate and 2 epochs has an accuracy of 57.41.
Model with 1 neurons, 0.001-learning rate and 3 epochs has an accuracy of 57.48.
Model with 1 neurons, 0.001-learning rate and 4 epochs has an accuracy of 55.86.


__________

Видно, что на точность базовой NN влияют кол-во нейронов, learning rate и число эпох [не в учет кол-во слоёв]. Все эти показатели могут приводить к повышению точности предсказаний, хотя learning rate и число эпох могут ухудшить точность GD, если поставить слишком высокие значения